In [ ]:
%matplotlib osx
%load_ext autoreload
%autoreload 2
# from matplotlib import pyplot as plt
# # plt.rcParams["figure.figsize"] = (12, 5)
import numpy as np
# from xbout import open_boutdataset
from analysis import *
import os

In [ ]:
ds = read_boutdata(r"../data", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/MASTU_runs/n_scan/with_churn/n8", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/MASTU_runs/sepx_dance_run/data", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/MASTU_runs/q_targets/with_churn/n8", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/MASTU_runs/mpi_test/c1/without_churn/n8", remove_xgc = True, units="a_mid")

# Get the rolling average
ds_avg = ds.rolling(t=5, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]

In [ ]:
# animate_nulls(ds_avg, vmax=3,vmin=1,extend="both", savepath="nulls_evolution_high_p_noavg.mov")
animate_nulls(ds_avg, vmax=3,vmin=1,extend="both")

In [ ]:
lambda_sol = 0.004
a = 0.65
a * (lambda_sol/a)**(1/3)

In [ ]:
_ = explore_nulls(ds, t=-1, levels=500, vmin=0, vmax=2, extend="both")

# PlasmaPy stuff

In [ ]:
from astropy import units as u 
import plasmapy as pp 

Te = (200.0) * u.eV 
Ti = Te 
ne = 1.5e19 * u.m**-3
p = 2*ne*pp.formulary.braginskii.e*Te
print(p)
ni = ne
ion = "D+"
kappa = pp.formulary.electron_thermal_conductivity(Te, ne, Ti, ni, ion)
print(kappa)

In [ ]:
Te = (30.0) * u.eV 
Ti = Te 
ne = 1.0e19 * u.m**-3
ion = "D+"
pp.formulary.Coulomb_logarithm(Te,ne,species=["e-", "p+"])

In [ ]:
import astropy.units as u
import plasmapy as pp 

n = 1e19 * u.m**-3
T = 200 * u.eV
species = ("e", "p")
r = pp.formulary.Spitzer_resistivity(T, n, species)  

In [ ]:
c = 3e8
1e-6 / (4 * np.pi * 0.5**2 / (c**2 * (1e-9*r/9)))

# Original

In [ ]:
plot_lineslice(ds, variable="P", timesteps=[-1], levels=300)

In [ ]:
ds.metadata["t_0"] / ds.metadata["n_sepx"]

In [ ]:
ds.metadata["P_0"] * 1.5

In [ ]:
a_mid = 0.2
lambda_sol = 0.008
print("{:.2f}cm".format(100*a_mid*(lambda_sol/a_mid)**(1/3)))

In [ ]:
##### ANIMATE LIST OF VARIABLES
ds.bout.animate_list(["P"],ncols=1, cmap="inferno", aspect="equal")
# ds.bout.animate_list([ds["q_par_y"].isel(y=-1)],ncols=1)

In [ ]:
a = ds.metadata["a_mid"]
r_cz = 20.98
u_avg = ds_avg_zoomed["u_mag"].integrate(coord="x").integrate(coord="y").isel(t=-1).values
u_x = 0.25 * u_avg / (np.sqrt(np.pi) * (23.6/(100*a))) 

In [ ]:
ds.bout.animate_list([ds["q_par_y"].isel(y=-1), ds["P"].isel(x=0), ds["P"].isel(y=0), ds["P"].isel(x=-1)], ncols=1)

In [ ]:
plot_q_targets(ds_avg, timestep=-1, show_t0=True, normalise=False, xaxis="flux")

In [ ]:
animate_q_targets(ds, normalise=False)

In [ ]:
Q_target_proportions(ds_avg, normalise=True)

In [ ]:
_, P1, P2, P3, P4 = get_Q_legs(ds_avg)

print(P1.values[-1])
print(P2.values[-1])
print(P3.values[-1])
print(P4.values[-1])

In [ ]:
plot_Q_targets(ds_avg, normalise=False)

In [ ]:
# Plot P_2/P_4
Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)

fig,ax = plt.subplots(1)
ax.plot(ds.t, Q2/Q4)
ax.set_xlabel("t [$t_0$]")
ax.set_ylabel("$P_2 / P_4$")
ax.grid()

In [ ]:
# ds_avg.bout.animate_list([ds_avg.metadata["C_s0"]*np.sqrt(ds_avg["u_x"]**2 + ds_avg["u_y"]**2)],ncols=1,cmap="inferno", logscale=False, vmax=3000)
ds.bout.animate_list([ds["P"].isel(y=60)],ncols=1, logscale=False)

In [ ]:
# Statistical equilibrium
# err = np.sqrt(((ds_avg["P"] - ds_avg["P"].shift(t=1))**2).integrate("x").integrate("y"))
err = abs(ds_avg["P"].differentiate("t").integrate("x").integrate("y"))
fig,ax = plt.subplots(1)
ax.plot(ds.t - ds.t.isel(t=0),err.values)
ax.set_xlabel(r"t [$t_0$]")
ax.set_ylabel(r"$\frac{\partial P}{\partial t}$")
ax.set_yscale("log")
ax.grid()

In [ ]:
predicted_r_cz(ds)

In [ ]:
# animate_vector(ds_zoomed.isel(t=range(1,len(ds.t))), "u", density=2.0, plot_every=1,arrowsize=0.5, lw_prefactor=5e1)
animate_vector(ds, "u", fps=8, density=0.3, plot_every=1, arrowsize=1, broken_streamlines=False)
# ntrim=30
# animate_vector(ds.isel(x=range(ntrim,len(ds.x)-ntrim), y=range(ntrim,len(ds.y)-ntrim)), "q_par", plot_every=10, savepath="q_par.gif")

In [ ]:
plot_vector(ds_avg, "u", t=-1, density=0.5, lw_prefactor=0.5, broken_streamlines=False)
# plot_vector(ds, "q_perp", t=-1, density=0.5, broken_streamlines=False, arrowsize=1.5, lw_prefactor=30000)
# plot_vector(ds_avg, "u", t=-1, density=0.5, broken_streamlines=False, arrowsize=0.0)

In [ ]:
animate_contour_list(ds_avg, vars=["psi"],max_timestep=None,fps=24,plot_r_cz=False, num_levels=100)

In [ ]:
fig,ax = plt.subplots(1)
ax.grid()
ax.axvline(np.median(ds.x))
ax.axhline(np.median(ds.y))
ax.set_xlim((np.min(ds.x),np.max(ds.x)))
ax.set_ylim((np.min(ds.y),np.max(ds.y)))
ax.contour(ds.x,ds.y,ds.isel(t=0).psi.values.T)

In [ ]:
contour_overlay(ds, "psi", [0], fill=False, colorbar=True, levels=30, extend="both")
# contour_overlay(ds, "B_p", [0], fill=True, colorbar=True, levels=np.linspace(0,0.05,10), extend="neither")
# contour_overlay(ds_zoomed, "psi", [0], fill=False, colorbar=True, extend="neither")
# contour_overlay(ds_zoomed, "psi", [-1], fill=False, colorbar=True, extend="neither")

In [ ]:
plot_conservation(ds)

In [ ]:
predicted_r_cz(ds) * ds.metadata["a_mid"] * 100

# 27/11/2024 analysis

In [ ]:
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_0p5/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_1p0/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_5p0/eps_0p00"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_0p5/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_1p0/eps_0p00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p00"]
rundirs = [r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_0p5/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_1p0/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/beta_p_scan_longrun/P_core_5p0/eps_0p00"]

P_cores = [0.5,1.0,5.0]
beta_ps = []
Q1s = []
Q2s = []
Q3s = []
Q4s = []

for rundir in rundirs:
    ds = read_boutdata(os.path.join(rundir, "data"), remove_xgc=True)
    # Get the rolling average
    ds_avg = ds.rolling(t=5, min_periods=1).mean()
    ds_avg_zoomed = ds_avg.isel(
        x=range(int(len(ds.x) / 4), 3 * int(len(ds.x) / 4)),
        y=range(int(len(ds.y) / 4), 3 * int(len(ds.y) / 4)),
    )

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds)
    Q_tot = Q1 + Q2 + Q3 + Q4
    Q1s.append(100*(Q1/Q_tot).isel(t=-1).values)
    Q2s.append(100*(Q2/Q_tot).isel(t=-1).values)
    Q3s.append(100*(Q3/Q_tot).isel(t=-1).values)
    Q4s.append(100*(Q4/Q_tot).isel(t=-1).values)

    beta_ps.append(ds.metadata["beta_p"])
beta_ps = [P_cores[i] * beta_ps[i] for i in range(len(P_cores))]

fig,ax = plt.subplots(1)
ax.stackplot(beta_ps, [Q1s,Q2s,Q3s,Q4s], labels=["Leg 1", "Leg 2", "Leg 3", "Leg 4"])
ax.legend()
ax.set_xlabel(r"$\beta_{pmid}$")
ax.set_ylabel(r"Power fraction [%]")

In [ ]:
ds1 = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_1p0/eps_0p00/data", remove_xgc = True)
ds2 = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_0p1/P_core_1p0/eps_0p00/data", remove_xgc = True)

Py1 = ds1["P"].isel(x=int(len(ds1.x)/2),t=0)
Py2 = ds2["P"].isel(x=int(len(ds2.x)/2),t=0)

r_cz1 = predicted_r_cz(ds1) * ds1.metadata["a_mid"]
r_cz2 = predicted_r_cz(ds2) * ds2.metadata["a_mid"]

midpoint = ds1.y.isel(y=int(len(ds1.y)/2)).values * ds1.metadata["a_mid"]

fig,ax = plt.subplots(1)
ax.plot(ds1.y * ds1.metadata["a_mid"],Py1, label=r"$\chi_{\perp}=1.0m^2s^-1$")
ax.plot(ds2.y * ds2.metadata["a_mid"],Py2, label=r"$\chi_{\perp}=0.1m^2s^-1$")
ax.legend()
ax.grid()
ax.set_ylabel("$P$ [$P_0$]")
ax.set_xlabel("$y$ [cm]")
ax.axvline(1.15*midpoint+r_cz1, linestyle="--", color="gray")
ax.axvline(1.15*midpoint-r_cz1, linestyle="--", color="gray")
fig.tight_layout()

In [ ]:
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs/with_churn/chi_perp_1p0/P_core_5p0/eps_0p08"]

r_seps = [0.0,7.0,14.0]
Q1s = []
Q2s = []
Q3s = []
Q4s = []
Q1s_0 = []
Q2s_0 = []
Q3s_0 = []
Q4s_0 = []

for rundir in rundirs:
    ds = read_boutdata(os.path.join(rundir, "data"), remove_xgc=True)
    # Get the rolling average
    ds_avg = ds.rolling(t=10, min_periods=1).mean()
    ds_avg_zoomed = ds_avg.isel(
        x=range(int(len(ds.x) / 4), 3 * int(len(ds.x) / 4)),
        y=range(int(len(ds.y) / 4), 3 * int(len(ds.y) / 4)),
    )

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds)
    Q_tot = Q1 + Q2 + Q3 + Q4
    Q1s.append(100*(Q1/Q_tot).isel(t=-1).values)
    Q2s.append(100*(Q2/Q_tot).isel(t=-1).values)
    Q3s.append(100*(Q3/Q_tot).isel(t=-1).values)
    Q4s.append(100*(Q4/Q_tot).isel(t=-1).values)
    Q1s_0.append(100*(Q1/Q_tot).isel(t=0).values)
    Q2s_0.append(100*(Q2/Q_tot).isel(t=0).values)
    Q3s_0.append(100*(Q3/Q_tot).isel(t=0).values)
    Q4s_0.append(100*(Q4/Q_tot).isel(t=0).values)

Q1s = np.array(Q1s)
Q2s = np.array(Q2s)
Q3s = np.array(Q3s)
Q4s = np.array(Q4s)
Q1s_0 = np.array(Q1s_0)
Q2s_0 = np.array(Q2s_0)
Q3s_0 = np.array(Q3s_0)
Q4s_0 = np.array(Q4s_0)

fig,ax = plt.subplots(1)
ax.plot(r_seps, Q3s/100, color='red', label="with churn")
ax.plot(r_seps, Q3s_0/100, color="black", label="without churn")
# ax2 = ax.twinx()
# ax2.plot(r_seps, Q2s / Q2s_0, color="gray", linestyle="--", label="ratio")
ax.legend()
# ax2.legend(loc="lower right")
# ax2.set_ylabel("ratio")
ax.set_xlabel(r"$r_{sep}$ [cm]")
ax.set_ylabel(r"Power fraction")
ax.grid()
ax.set_yscale("log")
ax.set_title("Leg 3")
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(3)
ax[0].plot(r_seps, Q2s/100, color='red', label="Leg 2 (=SP3)")
ax[1].plot(r_seps, Q3s/100, color='red', label="Leg 3 (=SP2)")
ax[2].plot(r_seps, Q4s/100, color='red', label="Leg 4 (=SP1)")
ax[0].plot(r_seps, Q2s_0/100, color='black', linestyle="--")
ax[1].plot(r_seps, Q3s_0/100, color='black', linestyle="--")
ax[2].plot(r_seps, Q4s_0/100, color='black', linestyle="--")
ax[0].grid()
ax[1].grid()
ax[2].grid()
# ax.plot(r_seps, Q3s_0/100, color="black", label="without churn")
ax[0].legend(loc="upper left")
ax[1].legend(loc="upper left")
ax[2].legend(loc="upper left")
# ax2.legend(loc="lower right")
# ax2.set_ylabel("ratio")
ax[-1].set_xlabel(r"$r_{sep}$ [cm]")
# ax.set_ylabel(r"Power fraction")
ax[0].set_yscale("log")
ax[1].set_yscale("log")
ax[0].set_ylim((0.01,0.3))
ax[1].set_ylim((0.01,0.2))
ax[2].set_ylim((0,0.7))
ax2 = ax[2].twinx()
ax2.plot([],[],color="red", label="With churn")
ax2.plot([],[],color="black", linestyle="--", label="Without churn")
ax2.legend(loc="lower left")
ax2.set_yticks([])
# ax[0].set_title("Leg 3")
fig.tight_layout()

# 4/12/2024 analysis

In [ ]:
# Topology change
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/topo_reconfig/TCV/P_0_30x", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=20, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

levels = np.array([-0.5,0.00005,0.00479,0.5])
contour_overlay(ds_avg, "psi", [0], fill=True, levels=levels)

# levels = np.array([-0.5,0.00166,0.00387,0.5])
levels = np.array([-0.5,0.00387,0.00891,0.5])
contour_overlay(ds_avg, "psi", [-1], fill=True, levels=levels)

In [ ]:
rundirs = [r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_10x",
           r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_30x",
           r"/Users/power8/Documents/07_churning_mode/topo_reconfig_investigation/TCV/P_0_50x"]

multipliers = np.array([10,30,50])
ratios_t0 = []
ratios_t1 = []
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=20, min_periods=1).mean()

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    ratios_t0.append((Q2 / Q4).isel(t=0).values)
    ratios_t1.append((Q2 / Q4).isel(t=-1).values)
    q_target_proportions(ds_avg, normalise=True)

ratios_t0 = np.array(ratios_t0)
ratios_t1 = np.array(ratios_t1)

fig,ax = plt.subplots(1)
# ax.plot(multipliers,ratios_t0,linestyle="--",marker="x")
ax.axhline(ratios_t0[0], linestyle="--", color="gray",label="inter ELM")
ax.plot(np.sqrt(multipliers), ratios_t1, linestyle="--", marker="x", label="ELM")
ax.set_ylabel("$P_3 / P_1$")
ax.set_xlabel(r"ELM $\beta_p$ multiplier")
ax.legend()
ax.grid()

# 11/12/2024 analysis

In [ ]:
# Match DIII-D experimental beta_p profiles
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_2/with_churn/sigma_01", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=10, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
ds_zoomed["B_p"] = np.sqrt(ds_zoomed["B_x"]**2 + ds_zoomed["B_y"]**2) * ds_zoomed.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)

lambda_sol = 0.003
D_expansion = ds.metadata["a_mid"] * (lambda_sol/ds.metadata["a_mid"])**(1/3)
f_exp = D_expansion/lambda_sol
print(f_exp)

timestamp = 0

colours = ["red", "blue", "green"]

fig,ax = plt.subplots(3)
offsets = ([3,2],[3,4],[3,6])
for i in range(len(offsets)):
    xcoord = int(len(ds.x)/2)+offsets[i][0]
    ycoord = int(len(ds.x)/2)+offsets[i][1]
    beta_p = ds["beta_p"].isel(t=timestamp, y=ycoord)
    P_e = 0.5 * 10 * ds.metadata["P_0"] * ds["P"].isel(t=timestamp, y=ycoord)
    P_b = ds["P_b"].isel(t=timestamp, y=ycoord)
    rho = ds["rho"].isel(t=timestamp, y=ycoord)
    xaxis = (ds.x - ds.x[xcoord]) * 10/f_exp
    # xaxis = -rho
    ax[0].scatter(xaxis,P_e, color=colours[i])
    ax[1].scatter(xaxis,P_b, color=colours[i])
    ax[2].scatter(xaxis,beta_p, color=colours[i])

    print("{}, x:{:.1f}, y:{:.1f}".format(colours[i],ds.x[int(len(ds.x)/2+1)],ds.y[ycoord]))

ax[0].set_yscale("log")
ax[0].set_ylim([500,3e5])
ax[0].set_xlim([-1,6])
ax[0].grid()
ax[0].set_ylabel("$P_e$ [dyn cm$^{-2}$]")

ax[1].set_yscale("log")
ax[1].set_ylim([100,10000])
ax[1].set_xlim([-1,6])
ax[1].grid()
ax[1].set_ylabel("$P_b$ [dyn cm$^{-2}$]")

ax[2].set_yscale("log")
ax[2].set_ylim([0.1,400])
ax[2].set_xlim([-1,6])
ax[2].grid()
ax[2].set_ylabel(r"$\beta_p$")

In [ ]:
# Match DIII-D EFIT beta_p profiles
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_2/with_churn/sigma_01", remove_xgc = True)
# ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01", remove_xgc = True)

# Get the rolling average
ds_avg = ds.rolling(t=10, min_periods=1).mean()
ds_avg_zoomed = ds_avg.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))
ds_zoomed = ds.isel(x=range(int(len(ds.x)/4), 3*int(len(ds.x)/4)), y=range(int(len(ds.y)/4), 3*int(len(ds.y)/4)))

ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
ds_zoomed["B_p"] = np.sqrt(ds_zoomed["B_x"]**2 + ds_zoomed["B_y"]**2) * ds_zoomed.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 100 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)

# timestamp = 0
timestamp = 0

colours = ["red", "blue", "green"]

fig,ax = plt.subplots(3)
offsets = [[4,2]]
for i in range(len(offsets)):
    xcoord = int(len(ds.x)/2)+offsets[i][0]
    ycoord = int(len(ds.y)/2)+offsets[i][1]
    beta_p = ds["beta_p"].isel(t=timestamp, x=xcoord)
    P_e = 0.5 * 10 * ds.metadata["P_0"] * ds["P"].isel(t=timestamp, x=xcoord)
    P_b = ds["P_b"].isel(t=timestamp, x=xcoord)
    rho = ds["rho"].isel(t=timestamp, x=xcoord)
    xaxis = ds.y
    # xaxis = -rho
    ax[0].scatter(xaxis,P_e, color=colours[i])
    ax[1].scatter(xaxis,P_b, color=colours[i])
    ax[2].scatter(xaxis,beta_p, color=colours[i])

    print("{}, x:{:.1f}, y:{:.1f}".format(colours[i],ds.x[xcoord],ds.y[ycoord]))

ax[0].set_yscale("log")
ax[0].set_ylim([0.7e4,7e5])
ax[0].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[0].grid()
ax[0].set_ylabel("$P_e$ [dyn cm$^{-2}$]")

ax[1].set_yscale("log")
ax[1].set_ylim([50,9e3])
ax[1].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[1].grid()
ax[1].set_ylabel("$P_b$ [dyn cm$^{-2}$]")

ax[2].set_yscale("log")
ax[2].set_ylim([10,400])
ax[2].set_xlim([ds.y[int(len(ds.y)/2)]-4.0,ds.y[int(len(ds.y)/2)]+25])
ax[2].grid()
ax[2].set_ylabel(r"$\beta_p$")
ax[2].set_xlabel("y [cm]")

In [ ]:
ds.bout.animate_list(["P_b"],logscale=True)

In [ ]:
# Angle change
rundirs = [r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p0",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p2",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p4",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p6",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_0p8",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p0",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p2",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p4",
           r"/Users/power8/Documents/07_churning_mode/angle_change/with_churn/theta_1p6"]

# x1_is = [[57,66], [57.2,68.3]]
# x2_is = [[83.6,66], [83.5,63.7]]
# x1_fs = [[81.7,66.7], [86.8,59.2]]
# x2_fs = [[54.9,66.6], [54.3,69.9]]

Q1_is =[]
Q2_is =[]
Q3_is =[]
Q4_is =[]
Q1_fs =[]
Q2_fs =[]
Q3_fs =[]
Q4_fs =[]
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=10, min_periods=1).mean()

    Qin, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Qtot = Q1 + Q2 + Q3 + Q4
    Q1_is.append((Q1/Qtot).isel(t=0))
    Q2_is.append((Q2/Qtot).isel(t=0))
    Q3_is.append((Q3/Qtot).isel(t=0))
    Q4_is.append((Q4/Qtot).isel(t=0))
    Q1_fs.append((Q1/Qtot).isel(t=-1))
    Q2_fs.append((Q2/Qtot).isel(t=-1))
    Q3_fs.append((Q3/Qtot).isel(t=-1))
    Q4_fs.append((Q4/Qtot).isel(t=-1))


In [ ]:
angles = [0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6]

fig,ax = plt.subplots(4, sharex=True)
ax[0].plot(angles, Q1_is, color="black", linestyle="--")
ax[0].plot(angles, Q1_fs, color="red", linestyle="--")
ax[0].set_ylim([1e-1,1.0])
ax[0].set_yscale("log")
ax[0].set_ylabel("$P_1$")

ax[1].plot(angles, Q2_is, color="black", linestyle="--")
ax[1].plot(angles, Q2_fs, color="red", linestyle="--")
ax[1].set_ylim([1e-4,0.1])
ax[1].set_yscale("log")
ax[1].set_ylabel("$P_2$")

ax[2].plot(angles, Q3_is, color="black", linestyle="--")
ax[2].plot(angles, Q3_fs, color="red", linestyle="--")
ax[2].set_ylim([1e-4,0.1])
ax[2].set_yscale("log")
ax[2].set_ylabel("$P_3$")

ax[3].plot(angles, Q4_is, color="black", linestyle="--")
ax[3].plot(angles, Q4_fs, color="red", linestyle="--")
ax[3].set_ylim([1e-1,1.0])
ax[3].set_yscale("log")
ax[3].set_xlabel(r"$\theta$")
ax[3].set_ylabel("$P_4$")


In [ ]:
# Replicate fig 6 in DIII-D paper (SP3,2,1 power fraction vs r_sep)
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_03"]
        #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_04",
        #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_05"]


# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])
# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0])
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])
P1s = []
P2s = []
P3s = []
timestamp = -1
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds_avg = ds.rolling(t=100, min_periods=1).mean()

    _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Q_tot = Q1 + Q2 + Q3 + Q4
    P1s.append((Q1/Q_tot).isel(t=timestamp))
    P2s.append((Q2/Q_tot).isel(t=timestamp))
    P3s.append((Q3/Q_tot).isel(t=timestamp))

fig,ax = plt.subplots(3,sharex=True)
ax[0].plot(r_sep,P3s, color="red", marker="x", linestyle="--")
ax[0].set_yscale("log")
ax[0].set_ylim([0.0095,0.11])
ax[0].set_ylabel("$P_3$")
ax[0].grid()
ax[1].plot(r_sep,P2s, color="red", marker="x", linestyle="--")
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.25])
ax[1].set_ylabel("$P_2$")
ax[1].grid()
ax[2].plot(r_sep,P1s, color="red", marker="x", linestyle="--")
ax[2].set_ylim([0.0,0.71])
ax[2].set_ylabel("$P_1$")
ax[2].set_xlim([-1,30])
ax[2].grid()
ax[2].set_xlabel("$D_{xx}$ [cm]")

In [ ]:
# Time series of power to each leg vs. D_xx
rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_01",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_02",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_03",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_04",
           r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_05"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_03",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_03",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_04",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_05"]
# rundirs = [r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_00",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_01",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_02",
#            r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_03"]
#         #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_04",
#         #    r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_lowP/with_churn/sigma_05"]

# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5, 2.0,2.5])
# r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])

# timestamps = [0,50,100,150,200]
timestamps = [-100, -50,-1]
tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn/sigma_00", remove_xgc = True).t

P1s = []
P2s = []
P3s = []
P4s = []
for rundir in rundirs:
    ds = read_boutdata(rundir, remove_xgc = True)
    ds = ds.interp(t=tgrid)
    ds_avg = ds.rolling(t=50, min_periods=1).mean()

    _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
    Q_tot = Q1 + Q2 + Q3 + Q4
    P1s.append([])
    P2s.append([])
    P3s.append([])
    P4s.append([])
    for timestamp in timestamps:
        P1s[-1].append((Q1/Q_tot).isel(t=timestamp))
        P2s[-1].append((Q2/Q_tot).isel(t=timestamp))
        P3s[-1].append((Q3/Q_tot).isel(t=timestamp))
        P4s[-1].append((Q4/Q_tot).isel(t=timestamp))

P1s = np.array(P1s).T
P2s = np.array(P2s).T
P3s = np.array(P3s).T
P4s = np.array(P4s).T

alphas = np.linspace(0.25,1.0,len(timestamps))

fig,ax = plt.subplots(4,sharex=True)
for i in range(len(timestamps)):
    ax[0].plot(r_sep,P4s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(timestamps)):
    ax[1].plot(r_sep,P3s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(timestamps)):
    ax[2].plot(r_sep,P2s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(timestamps)):
    ax[3].plot(r_sep,P1s[i], color="red", marker="x", linestyle="--", alpha=alphas[i])
ax[3].set_ylim([0.0,0.71])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("$D_{xx}$ [cm]")

for i in range(len(timestamps)):
    ax[0].plot([],[],color="red",linestyle="--",marker="x",alpha=alphas[i],label="t={:.2f}ms".format(1000* (tgrid[timestamps[i]]-tgrid[0]) * ds.metadata["t_0"]))
ax[0].legend(loc="upper right")

In [ ]:
# Plot psi using parameters from EFIT fit
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_psi/data", remove_xgc = True)
fig,ax = plt.subplots(1)
levels = np.linspace(0.031415719597447156, 0.052115647479612406,50)
# levels=50
c = ax.contour(ds.x, ds.y, ((ds["psi"]*ds.metadata["psi_0"]) + (1.07e-03*3.91e+01)).isel(t=-1).values.T, levels=levels, cmap="inferno")
# c = ax.pcolormesh(ds.x, ds.y, ((ds["psi"]*ds.metadata["psi_0"]) + (1.07e-03*3.91e+01)).isel(t=-1).values.T, cmap="inferno")
fig.colorbar(c)
ax.set_xlabel("R [m]")
ax.set_ylabel("Z [m]")

In [ ]:
# Plot pressure along same line as DTS EFIT measurement in DIII-D paper (fig. 11)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_P/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True, units="a_mid")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02", remove_xgc = True)
# ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
# mu_0 = 1.256637e-6
# ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)
ds["P_el"] = 0.5 * 10 * ds["P"] * ds.metadata["P_0"]
ds["x"] = (ds.x-np.median(ds.x))
ds["y"] = (ds.y-np.median(ds.y))
# ds = ds.rolling(t=50,min_periods=1).mean()

fig,ax = plt.subplots(ncols=2,nrows=1)

alpha = 0.25
x_rot = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_rot = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)

ds_rot = ds.interp(x=x_rot,y=y_rot)
timestamp = -1
# timestamp = 0
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=[-0.000353,0], colors="white")
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.04199,-0.041635,2), colors="white")
ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.00035,0.00,2), colors="white")
ax[0].pcolormesh(ds_rot.x,ds_rot.y,ds_rot["P"].isel(t=timestamp).values.T, cmap="inferno")
# x_idx = find_nearest(ds_rot.x, 7.0)
# y_idx = find_nearest(ds_rot.y, -2.5)
x_1 = 0.093 # x-coordinate of first X-point
y_1 = -0.038 # y-coordinate of first X-point
x_idx = find_nearest(ds_rot.x, x_1)
y_idx = find_nearest(ds_rot.y, y_1)
ax[0].axvline(x=ds_rot.x.isel(x=x_idx), color="red")

ax[1].plot(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y))),ds_rot["P_el"].isel(x=x_idx,t=timestamp,y=range(y_idx,len(ds_rot.y))), color="red", marker="x")
ax[1].set_xlabel("y [cm]")
ax[1].set_ylabel("$P_{el}$")
ax[1].set_yscale("log")
ax[1].grid()
# ax[1].set_ylim([0.8e4,0.7e6])
# ax[1].set_xlim(min(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y)))),min(ds_rot.y.isel(y=range(y_idx,len(ds_rot.y))))+27)
# ax[1].axvline(x=7.0,linestyle="--",color="gray")
fig.tight_layout()

# 18/12/2024 analysis

In [ ]:
# Plot P_l vs. D_xx for different null orientations
sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03"]
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5])

# th1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3/with_churn"
# th2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3_theta2/with_churn"
# th3_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_3_theta3/with_churn"
# rundirs  = [th1_rundir, th2_rundir, th3_rundir]
th1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
th2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn"
th3_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_150/with_churn"
th4_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_m025/with_churn"
th5_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_m075/with_churn"
rundirs  = [th1_rundir, th2_rundir, th3_rundir, th4_rundir, th5_rundir]
thetas = [r"$\theta=14^{\degree}$", r"$\theta=43^{\degree}$", r"$\theta=86^{\degree}$", r"$\theta=-14^{\degree}$", r"$\theta=-43^{\degree}$"]
# rundirs  = [th1_rundir]
# thetas = [r"$\theta=14^{\degree}$"]

tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn/sigma_00", remove_xgc = True).t

markers=["x", "+", "*", "o", "v"]

timestamp = 0
avg_window = 50
P1s = np.zeros((len(rundirs),len(sigmas)))
P2s = np.zeros((len(rundirs),len(sigmas)))
P3s = np.zeros((len(rundirs),len(sigmas)))
P4s = np.zeros((len(rundirs),len(sigmas)))
for i,rundir in enumerate(rundirs):
    for j,sigma in enumerate(sigmas):
        ds = read_boutdata(os.path.join(rundir,sigma), remove_xgc = True)
        ds.interp(t=tgrid)
        ds_avg = ds.rolling(t=avg_window, min_periods=1).mean()

        _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
        Q_tot = Q1 + Q2 + Q3 + Q4
        P1s[i,j] = (Q1/Q_tot).isel(t=timestamp)
        P2s[i,j] = (Q2/Q_tot).isel(t=timestamp)
        P3s[i,j] = (Q3/Q_tot).isel(t=timestamp)
        P4s[i,j] = (Q4/Q_tot).isel(t=timestamp)

alphas = np.linspace(1.0,0.25,len(rundirs))

# fig,ax = plt.subplots(3,sharex=True)
fig,ax = plt.subplots(4,sharex=True)
for i in range(len(rundirs)):
    ax[0].plot(r_sep,P4s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(rundirs)):
    ax[1].plot(r_sep,P3s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
# ax[1].set_ylim([0.0065,0.3])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(rundirs)):
    ax[2].plot(r_sep,P2s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(rundirs)):
    ax[3].plot(r_sep,P1s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[3].set_ylim([-0.03,0.73])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("Inter-null separation [cm]")

for i in range(len(rundirs)):
    ax[0].plot([],[],color="red", marker=markers[i], linestyle="--", alpha=alphas[i], label=thetas[i])
ax[0].legend(loc="upper right")

In [ ]:
# Plot P_l vs. D_xx for different null orientations
sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03", "sigma_04", "sigma_05"]
r_sep = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])

res1_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
res2_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn"
rundirs  = [res1_rundir, res2_rundir]
resolutions = [r"64x40 cells", r"256x160 cells"]

tgrid = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_00", remove_xgc = True).t

markers=["x", "+", "*", "o", "v"]

# timestamp = -1
# avg_window = 50
# P1s = np.zeros((len(rundirs),len(sigmas)))
# P2s = np.zeros((len(rundirs),len(sigmas)))
# P3s = np.zeros((len(rundirs),len(sigmas)))
# P4s = np.zeros((len(rundirs),len(sigmas)))
# for i,rundir in enumerate(rundirs):
#     for j,sigma in enumerate(sigmas):
#         ds = read_boutdata(os.path.join(rundir,sigma), remove_xgc = True)
#         ds.interp(t=tgrid)
#         ds_avg = ds.rolling(t=avg_window, min_periods=1).mean()

#         _, Q1, Q2, Q3, Q4 = get_Q_legs(ds_avg)
#         Q_tot = Q1 + Q2 + Q3 + Q4
#         P1s[i,j] = (Q1/Q_tot).isel(t=timestamp)
#         P2s[i,j] = (Q2/Q_tot).isel(t=timestamp)
#         P3s[i,j] = (Q3/Q_tot).isel(t=timestamp)
#         P4s[i,j] = (Q4/Q_tot).isel(t=timestamp)

alphas = np.linspace(1.0,0.25,len(rundirs))

fig,ax = plt.subplots(4,sharex=True)
for i in range(len(rundirs)):
    ax[0].plot(r_sep,P4s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[0].set_ylim([0.0,0.71])
ax[0].set_ylabel("$P_4$")
ax[0].set_xlim([-1,30])
ax[0].grid()
for i in range(len(rundirs)):
    ax[1].plot(r_sep,P3s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[1].set_yscale("log")
ax[1].set_ylim([0.0095,0.11])
# ax[1].set_ylim([0.0065,0.3])
ax[1].set_ylabel("$P_3$")
ax[1].grid()
for i in range(len(rundirs)):
    ax[2].plot(r_sep,P2s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[2].set_yscale("log")
ax[2].set_ylim([0.0095,0.25])
ax[2].set_ylabel("$P_2$")
ax[2].grid()
for i in range(len(rundirs)):
    ax[3].plot(r_sep,P1s[i,:], color="red", linestyle="--", marker=markers[i], alpha=alphas[i])
ax[3].set_ylim([-0.03,0.73])
ax[3].set_ylabel("$P_1$")
ax[3].set_xlim([-1,30])
ax[3].grid()
ax[3].set_xlabel("Inter-null separation [cm]")

for i in range(len(rundirs)):
    ax[0].plot([],[],color="red", marker=markers[i], linestyle="--", alpha=alphas[i], label=resolutions[i])
ax[0].legend(loc="upper right")

In [ ]:
# Plot psi along same line as DTS EFIT measurement in DIII-D paper (fig. 11)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/EFIT_comparison/DIII-D_fitted_P/data", remove_xgc = True)
ds = read_boutdata(r"/Users/power8/Documents/01_code/04_bout/churning_mode/data", remove_xgc = True)
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_hires/with_churn/sigma_02", remove_xgc = True)
ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
mu_0 = 1.256637e-6
ds["P_b"] = 10 * ((ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]**2) / (2*mu_0)
ds["P_el"] = 0.5 * 10 * ds["P"] * ds.metadata["P_0"]
ds["x"] = (ds.x-np.median(ds.x))
ds["y"] = (ds.y-np.median(ds.y))
ds = ds.rolling(t=50,min_periods=1).mean()

fig,ax = plt.subplots(ncols=2,nrows=1)

alpha = 0.25
x_rot = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_rot = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)

ds_rot = ds.interp(x=x_rot,y=y_rot)
timestamp = -1
# timestamp = 0
# ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=[-0.000353,0], colors="white")
ax[0].contour(ds_rot.x,ds_rot.y,ds_rot["psi"].isel(t=timestamp).values.T, levels=np.linspace(-0.04199,-0.041635,2), colors="white")
ax[0].pcolormesh(ds_rot.x,ds_rot.y,ds_rot["P"].isel(t=timestamp).values.T, cmap="inferno")
# x_idx = find_nearest(ds_rot.x, 7.0)
# y_idx = find_nearest(ds_rot.y, -2.5)
x_1 = 0.0070 # x-coordinate of first X-point  [a_mid]
y_1 = -0.0122 # y-coordinate of first X-point  [a_mid]
x_idx = find_nearest(ds_rot.x, x_1)
y_idx = find_nearest(ds_rot.y, y_1)
ax[0].axvline(x=ds_rot.x.isel(x=x_idx), color="red")
ax[0].axhline(y=ds_rot.y.isel(y=y_idx), color="blue")


psi_ds = ds_rot.isel(x=x_idx,t=timestamp,y=range(y_idx,len(ds_rot.y)))["psi"]
psi_ds["y"] = (ds_rot.y.isel(y=range(y_idx,len(ds_rot.y)))-ds_rot.y.isel(y=y_idx)-125)/100
ax[1].plot(psi_ds.y,psi_ds, color="red", marker="x")
ax[1].set_xlabel("y [cm]")
ax[1].set_ylabel("$psi$")
ax[1].grid()
# ax[1].axvline(x=5.5,linestyle="--",color="gray")
fig.tight_layout()


fig11_coords = [-1.2453719008264463, -1.2322314049586778,-1.2076859504132231,-1.1707438016528926,-1.1310743801652894,-1.0921487603305784,-1.0544628099173554,-1.0118181818181817]
psi_fig11 = psi_ds.interp(y=fig11_coords).values
print(psi_fig11)

In [ ]:
# Compare umbrella diffusion method vs turbulent calculation

sigmas = ["sigma_00", "sigma_01", "sigma_02", "sigma_03", "sigma_04", "sigma_05"]
r_seps = 11.4 * np.array([0.1,0.5,1.0,1.5,2.0,2.5])
# sigmas = ["sigma_00", "sigma_01"]
# r_seps = 11.4 * np.array([0.1,0.5])

turb_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4/with_churn"
diff_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_umbrella/theta_024/with_churn"
# turb_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_thetas/theta_075/with_churn"
# diff_rundir = r"/Users/power8/Documents/07_churning_mode/DIII-D_runs_4_umbrella/theta_075/with_churn"
rundirs  = [turb_rundir, diff_rundir]
resolutions = ["turbulent calc", "umbrella diff"]

plot_power_deposition_vs_d_sep(rundirs=rundirs, 
                               rundir_labels = resolutions, 
                               sepdirs = sigmas, 
                               seps=r_seps, 
                               rundir_colours=["red", "black"],
                               plot_p1=False,
                               plot_p4=False)

# 12/02/2025 analysis

In [ ]:
# Plot r_cz predicted vs. actual
a_mid = 0.65 
R_0 = 0.85
B_pmid = 0.3
B_pmid_corr = 0.4
p_null = np.array([500,1000,1500])
beta_p = 2 * mu_0 * p_null / B_pmid**2 
beta_p_corr = 2 * mu_0 * p_null / B_pmid_corr**2 
r_actual = np.array([7.0,9.0,11.5])
r_actual_errors = 3*np.ones(len(r_actual))
beta_p_predicted = np.linspace(min(beta_p), max(beta_p), 100)
beta_p_predicted_corr = np.linspace(min(beta_p_corr), max(beta_p_corr), 100)
r_predicted = 0.81 * a_mid * (beta_p_predicted * a_mid / R_0) **(1/3) * 100
r_predicted_corr = 0.81 * a_mid * (beta_p_predicted_corr * a_mid / R_0) **(1/3) * 100

fig,ax = plt.subplots()
ax.errorbar(beta_p, r_actual, r_actual_errors, marker="x", linestyle="--", label="Observed")
ax.plot(beta_p_predicted, r_predicted, label=r"$0.81a_{mid}(\beta_{pm} \varepsilon)^{1/3}$")
# ax.plot(beta_p_predicted, r_predicted_corr, label=r"$0.81a_{mid}(\beta_{pm}^* \varepsilon)^{1/3}$")
ax.set_ylabel("$r_{cz}$ [cm]")
ax.set_xlabel(r"$\beta_{pm}$")
ax.grid()
ax.legend()

In [ ]:
# Double check that 2nd order expansion of psi gives B_p = B_pm at (a_mid, sqrt(3)*a_mid)
a_mid = 0.8
R_0 = 1.5
B_pm = 1.0

x_hat = (np.linspace(-0.9,0.9,1000))
y_hat = np.linspace(-0.9,0.9,1000)
x_hat,y_hat = np.meshgrid(x_hat,y_hat)
x_hat = x_hat.T 
y_hat = y_hat.T
r_hat = 1 + (a_mid/R_0) * x_hat
z_hat = y_hat

psi_hat = (y_hat**3 - 3 * x_hat**2 * y_hat)
# psi = (y**3 - 3 * x**2 * y) * B_pm

def ddr(f, r):
    dfdr = np.zeros(f.shape)
    for i in range(1, f.shape[0] - 1):
        dfdr[i, :] = (f[i + 1, :] - f[i - 1, :]) / ((r[i + 1, :] - r[i - 1, :]))
    return dfdr


def ddz(f, z):
    dfdz = np.zeros(f.shape)
    for j in range(1, f.shape[1] - 1):
        dfdz[:, j] = (f[:, j + 1] - f[:, j - 1]) / ((z[:, j + 1] - z[:, j - 1]))
    return dfdz


Bx_hat = -(1 / r_hat) * ddz(psi_hat, z_hat)
By_hat = (1 / r_hat) * ddr(psi_hat, r_hat)
Bp_hat = np.sqrt(Bx_hat**2 + By_hat**2)
fig,ax = plt.subplots(1)
# ax.scatter([a_mid],[np.sqrt(3)*a_mid], color="red", marker="x", zorder=999)
# ax.scatter([0],[a_mid], color="red", marker="x", zorder=999)
ax.scatter([0],[np.sqrt(1/3)], color="blue", marker="x", zorder=999)
c = ax.contourf(x_hat,y_hat,Bp_hat, levels=[1 - 0.05,1 + 0.05])
# ax.plot(x[0,:],y_l, color="red")
fig.colorbar(c)

In [ ]:
# Find p_null for a given r_cz in MAST-U reference case

def p_null(r_cz: float, B_pm: float = 0.73, a_mid: float = 0.65, R_0: float = 0.85):
    p = (R_0 / a_mid) * ((r_cz / (0.81 * a_mid))**3) * (B_pm**2 / (2*mu_0))
    beta_p = (p * 2 * mu_0) / B_pm**2
    r_cz = 0.81 * a_mid * (beta_p * a_mid / R_0)**(1/3)
    # print(beta_p, r_cz)
    return p

def r_cz(B_pm: float = 0.3, p_null: float = 500, a_mid: float = 0.65, R_0: float = 0.85):
    beta_p = (p_null * 2 * mu_0) / B_pm**2
    return 0.81 * a_mid * (beta_p * a_mid / R_0) **(1/3)

def tau_ch(r_cz: float, R_0: float = 0.85, p_null: float= 500, n_sepx: float= 1e19):
    Ti = (0.5 * p_null / n_sepx) / el_charge
    v_th = np.sqrt(2*Ti*el_charge / m_i)
    return np.sqrt(R_0 * r_cz) / v_th


# p_null(0.1, B_pm = 0.4)
# r = r_cz(p_null = 500, B_pm=0.3)
r = 0.05
tau = tau_ch(p_null=1500, n_sepx=6e19, r_cz = r)
D_x = 0.5 * r**2 / tau
print(D_x)

In [ ]:
# Calculate quatities in n_sepx scan
runs = [r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_5cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_5p2cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_6cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_6p1cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_6p2cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_7cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_7p1cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_7p2cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_7p3cm",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/n_scan_SF75/with_churn/r_cz_7p4cm",]
d_sol = 0.0963292
nscan_data = extract_rundeck_data(runs, d_sol, avg_window=50)

In [ ]:
# Plot P_out vs. p_null
fig,ax = plt.subplots(1)
ax.plot(nscan_data["p_null"], nscan_data["P_out_w"] * 1e-6, linestyle="--", marker="x", label="With MHD")
ax.plot(nscan_data["p_null"], nscan_data["P_out_wo"] * 1e-6, linestyle="--", marker="x", label="Without MHD")
ax.grid()
ax.set_xlabel(r"$p_{null}$ [Pa]")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
ax.legend()

In [ ]:
# Plot P_out_ch / P_out vs. p_null
fig,ax = plt.subplots(1)
x = nscan_data["p_null"] / nscan_data["p_null"][0]
ax.plot(x, (nscan_data["P_out_w"] - nscan_data["P_out_wo"]) / (nscan_data["P_out_w"]- nscan_data["P_out_wo"])[0], linestyle="--", marker="x", label="With MHD")
ax.grid()
ax.set_xlabel(r"$p_{null} / p_{null,0}$")
ax.set_ylabel(r"$P_{out} / P_{out,0}$")
ax.legend()
ax.set_xscale("log")
ax.set_yscale("log")

In [ ]:
# Plot P_out vs. r_cz
fig,ax = plt.subplots(1)
ax.plot(nscan_data["r_ch"]*100, nscan_data["P_out_w"] * 1e-6, linestyle="--", marker="x", label="With MHD")
ax.plot(nscan_data["r_ch"]*100, nscan_data["P_out_wo"] * 1e-6, linestyle="--", marker="x", label="Without MHD")
ax.grid()
ax.set_xlabel(r"$r_{cz}$ [cm]")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
ax.legend()

In [ ]:
# Plot P_out vs. beta_p
fig,ax = plt.subplots(1)
ax.plot(nscan_data["beta_p"], nscan_data["P_out_w"] * 1e-6, linestyle="--", marker="x", label="With MHD")
ax.plot(nscan_data["beta_p"], nscan_data["P_out_wo"] * 1e-6, linestyle="--", marker="x", label="Without MHD")
ax.grid()
ax.set_xlabel(r"$\beta_p$")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
ax.legend()

In [ ]:
# Plot <chi_perp_ch> vs. p_null
fig,ax = plt.subplots(1)
ax.plot(nscan_data["p_null"], nscan_data["chi_perp_eff_w"] - nscan_data["chi_perp_eff_wo"], linestyle="-", color="black", marker="x")
ax.grid()
ax.set_xlabel(r"$p_{null}$ [Pa]")
ax.set_ylabel(r"$\langle \chi_{\perp}^{ch}\rangle$ [m$^2$s$^{-1}$]")
ax.plot([],[],color="black",linestyle="-",marker="x",label=r"$\langle \chi_{\perp}^{ch}\rangle$")
ax.legend()

In [ ]:
# Scaling of chi_x
a_mid = 0.65
fig,ax = plt.subplots(1)
x = nscan_data["p_null"] / np.sqrt(nscan_data["n_sepx"])
a = 500000000.0
c = -103
y = a * x + c
ax.plot(x, (nscan_data["chi_perp_eff_w"] - nscan_data["chi_perp_eff_wo"])* (a_mid**2 / (np.sqrt(np.pi)*(nscan_data["r_ch"]))), linestyle="-", color="black", marker="x", label=r"$\chi_x$")
ax.plot(x, y, linestyle="--", label=r"$\chi_x \propto p_{null}/\sqrt{n_e}$")
ax.set_xlabel(r"$p_{null}/\sqrt{n_e}$")
ax.set_ylabel(r"$\chi_x$ [m$^2$s$^{-1}$]")
ax.grid()
ax.set_ylim([0,None])
# ax.set_yscale("log")
ax.legend()

In [ ]:
# Plot observed vs predicted chi_x and chi_x vs. p_null
a_mid = 0.65
fig,ax = plt.subplots(1)
ax2 = ax.twinx()
ax2.plot(nscan_data["p_null"], nscan_data["D_x_th"], linestyle="--", color="red", marker="x")
ax.plot(nscan_data["p_null"], (nscan_data["chi_perp_eff_w"] - nscan_data["chi_perp_eff_wo"]) * (a_mid**2 / (np.sqrt(np.pi)*(nscan_data["r_ch"]))), linestyle="-", color="black", marker="x")
ax.grid()
ax.set_xlabel(r"$p_{null}$ [Pa]")
ax2.set_ylabel(r"Predicted $\chi_x$ [m$^2$s$^{-1}$]")
ax.set_ylabel(r"Observed $\chi_x$ [m$^2$s$^{-1}$]")
ax.plot([],[],color="black",linestyle="-",marker="x",label=r"Observed $\chi_x$")
ax.plot([],[],color="red",linestyle="--",marker="x",label=r"Predicted $\chi_x$")
ax2.spines['right'].set_color('red')
ax2.yaxis.label.set_color('red')
ax2.tick_params(axis='y', colors='red')
ax.legend()
fig.tight_layout()

In [ ]:
# Drift velocity across X-point
a_mid = 0.65
fig,ax = plt.subplots(1)
ax.plot(nscan_data["p_null"], ((nscan_data["P_out_w"] - nscan_data["P_out_wo"]) / a_mid) / nscan_data["p_null"], linestyle="-", color="black", marker="x")
ax.grid()
ax.set_xlabel(r"$p_{null}$ [Pa]")
ax.set_ylabel(r"$v_x$ [ms$^{-1}$]")
# ax.plot([],[],color="black",linestyle="-",marker="x",label=r"Observed $\chi_x$")
# ax.plot([],[],color="red",linestyle="--",marker="x",label=r"Predicted $\chi_x$")
# ax.legend()
fig.tight_layout()

In [ ]:
# Drift velocity across X-point
a_mid = 0.65
fig,ax = plt.subplots(1)
ax.plot((nscan_data["P_out_w"] - nscan_data["P_out_wo"]) / nscan_data["p_null"], ((nscan_data["P_out_w"] - nscan_data["P_out_wo"]) / a_mid) / nscan_data["p_null"], linestyle="-", color="black", marker="x")
ax.grid()
ax.set_xlabel(r"$p_{null}$ [Pa]")
ax.set_ylabel(r"$v_x$ [ms$^{-1}$]")
fig.tight_layout()

In [ ]:
# Plot p vs. r for exponential^2 decay
r = np.linspace(0,30)
p0 = 1500
d_sol = 10
p = p0 * np.exp(-(r/d_sol)**2)

fig,ax = plt.subplots()
ax.plot(r,p)

In [ ]:
# Calculate quatities in d_xx scan
runs = [r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_2p5cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_5p0cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_7p5cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_10p0cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_12p5cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_15p0cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_17p5cm",
        r"/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_20p0cm"]
d_sol = 0.0963292
dxxscan_data = extract_rundeck_data(runs, d_sol, avg_window=50)

In [ ]:
fig,ax = plt.subplots(1)
ax.plot(dxxscan_data["d_xx"] * 100, dxxscan_data["P_out_w"] * 1e-6, label="With MHD", linestyle="--", marker="x")
ax.plot(dxxscan_data["d_xx"] * 100, dxxscan_data["P_out_wo"] * 1e-6, label="Without MHD", linestyle="--", marker="x")
# ax.plot(dxxscan_data["d_xx"] * 100, (dxxscan_data["P_out_w"] - dxxscan_data["P_out_wo"]) * 1e-6, label="With MHD", linestyle="--", marker="x")
ax.legend()
ax.grid()
ax.set_xlabel("$d_{xx}$ [cm]")
ax.set_ylabel("$P_{out}$ [MWm$^{-1}$]")

In [ ]:
a_mid = 0.65
x = dxxscan_data["d_xx"] / a_mid
P_1 = dxxscan_data["P_legs_w"][:,3] / np.sum(dxxscan_data["P_legs_w"],axis=1)
P_3 = dxxscan_data["P_legs_w"][:,1] / np.sum(dxxscan_data["P_legs_w"],axis=1)
fig, ax = plt.subplots(1)
ax.plot(x,100*P_1,marker="x", linestyle="--", label="Leg 4", color="blue")
ax.plot(x,100*P_3,marker="x", linestyle="--", label="Leg 2", color="red")
ax.set_ylim([0,100])
ax.grid()
ax.legend()
ax.set_xlabel("$\sigma$")
ax.set_ylabel("Power fraction [%]")

In [ ]:
# Calculate quantities in d_sol scal (n=5e19)
runs = [r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_10e19/with_churn/d_sol_8cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_10e19/with_churn/d_sol_9cm",
        r"/Users/power8/Documents/07_churning_mode/old_rundecks/n_scan_SF75/with_churn/r_cz_7p4cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_10e19/with_churn/d_sol_11cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_10e19/with_churn/d_sol_12cm"]
d_sol = [0.0863,0.090654,0.0963292,0.1082924,0.121406]
dsolscan10 = extract_rundeck_data(runs, d_sol, avg_window=50)

In [ ]:
# Plot P_out vs. p_null
fig,ax = plt.subplots(1)
ax.plot([1/d for d in d_sol], (dsolscan10["P_out_w"] - dsolscan10["P_out_wo"]) * 1e-6, linestyle="--", marker="x", label="With MHD")
# ax.plot([1/d for d in d_sol], dsolscan7["P_out_wo"] * 1e-6, linestyle="--", marker="x", label="Without MHD")
ax.grid()
ax.set_xlabel(r"$1/d_{SOL}$ [m$^{-1}$]")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
# ax.legend()

In [ ]:
# Calculate quantities in d_sol scal (n=7e19)
runs = [r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_7e19/with_churn/d_sol_8cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_7e19/with_churn/d_sol_9cm",
        r"/Users/power8/Documents/07_churning_mode/old_rundecks/n_scan_SF75/with_churn/r_cz_7p1cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_7e19/with_churn/d_sol_11cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_7e19/with_churn/d_sol_12cm"]
d_sol = [0.0863,0.090654,0.0963292,0.1082924,0.121406]
dsolscan7 = extract_rundeck_data(runs, d_sol, avg_window=50)

In [ ]:
# Plot P_out vs. p_null
fig,ax = plt.subplots(1)
ax.plot([1/d for d in d_sol], (dsolscan7["P_out_w"] - dsolscan7["P_out_wo"]) * 1e-6, linestyle="--", marker="x", label="With MHD")
# ax.plot([1/d for d in d_sol], dsolscan7["P_out_wo"] * 1e-6, linestyle="--", marker="x", label="Without MHD")
ax.grid()
ax.set_xlabel(r"$1/d_{SOL}$ [m$^{-1}$]")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
# ax.legend()

In [ ]:
# Calculate quantities in d_sol scal (n=5e19)
runs = [r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_5e19/with_churn/d_sol_8cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_5e19/with_churn/d_sol_9cm",
        r"/Users/power8/Documents/07_churning_mode/old_rundecks/n_scan_SF75/with_churn/r_cz_6p2cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_5e19/with_churn/d_sol_11cm",
        r"/Users/power8/Documents/07_churning_mode/d_sol_scans/d_sol_scan_n_5e19/with_churn/d_sol_12cm"]
d_sol = [0.0863,0.090654,0.0963292,0.1082924,0.121406]
dsolscan5 = extract_rundeck_data(runs, d_sol, avg_window=50)

In [ ]:
# Plot P_out vs. p_null
fig,ax = plt.subplots(1)
ax.plot([1/d for d in d_sol], (dsolscan5["P_out_w"] - dsolscan5["P_out_wo"]) * 1e-6, linestyle="--", marker="x")
ax.grid()
ax.set_xlabel(r"$1/d_{SOL}$ [cm${^-1}$]")
ax.set_ylabel(r"$P_{out}$ [MWm$^{-1}$]")
# ax.legend()
# ax.set_yscale("log")

In [ ]:
dsolscan10["p_null"]

In [ ]:
fig,ax = plt.subplots(3, ncols=1, sharex=True)
ax[0].plot([1/d for d in d_sol], (dsolscan5["P_out_w"] - dsolscan5["P_out_wo"]) * 1e-6, linestyle="--", marker="x", label=r"$p_{null}\simeq 1200$ Pa")
ax[1].plot([1/d for d in d_sol], (dsolscan7["P_out_w"] - dsolscan7["P_out_wo"]) * 1e-6, linestyle="--", marker="x", label=r"$p_{null}\simeq 1600$ Pa")
ax[2].plot([1/d for d in d_sol], (dsolscan10["P_out_w"] - dsolscan10["P_out_wo"]) * 1e-6, linestyle="--", marker="x", label=r"$p_{null}\simeq 2300$ Pa")
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[0].legend(loc="upper left")
ax[1].legend(loc="upper left")
ax[2].legend(loc="upper left")
ax[2].set_xlabel(r"$1/d_{SOL}$ [cm$^{-1}$]")
ax[1].set_ylabel(r"$\Delta P_{out}$ [MWm$^{-1}$]")
fig.subplots_adjust(hspace=0)

# 19/02/2025 analysis

In [ ]:
# SF15 density scan
runs = ["/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_1e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_2e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_3e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_4e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_5e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_6e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_7e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_8e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_9e19",
        "/Users/power8/Documents/07_churning_mode/orientation_scan/n_scan_SF15/with_churn/n_10e19"]
sf15 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
# Show onset of topological change
ds1 = read_boutdata("/Users/power8/Documents/07_churning_mode/n_scan_SF15/with_churn/n_5e19")
ds_avg1 = ds1.rolling(t=10, min_periods=1).mean()
explore_nulls(ds1, t=0, vmin=0, vmax=3, levels=300)
explore_nulls(ds1, t=-1, vmin=0, vmax=3, levels=300)
# Q_target_proportions(ds_avg1.isel(t=[0,-1]), normalise=True)

ds2 = read_boutdata("/Users/power8/Documents/07_churning_mode/n_scan_SF15/with_churn/n_6e19")
ds_avg2 = ds2.rolling(t=10, min_periods=1).mean()
explore_nulls(ds2, t=0, vmin=0, vmax=3, levels=300)
explore_nulls(ds2, t=-1, vmin=0, vmax=3, levels=300)
# Q_target_proportions(ds_avg2.isel(t=[0,-1]), normalise=True)

In [ ]:
# Plot share of power to leg 2 and 3 vs. p_null 
x = sf15["p_null"]
P_2 = sf15["P_legs_w"][:,1] / np.sum(sf15["P_legs_w"],axis=1)
P_3 = sf15["P_legs_w"][:,2] / np.sum(sf15["P_legs_w"],axis=1)
fig, ax = plt.subplots(1)
ax.plot(x,100*P_2,marker="x", linestyle="--", label="Leg 2 (SE)")
ax.plot(x,100*P_3, marker="x", linestyle="--", label="Leg 3 (SW)")
ax.grid()
ax.legend()
ax.set_xlabel("$p_{null}$ [Pa]")
ax.set_ylabel("Power fraction [%]")

In [ ]:
# Plot ratio of power to leg 2 and 3 vs. p_null 
x = sf15["p_null"]
P_2 = sf15["P_legs_w"][:,1] / np.sum(sf15["P_legs_w"],axis=1)
P_3 = sf15["P_legs_w"][:,2] / np.sum(sf15["P_legs_w"],axis=1)
P_20 = sf15["P_legs_wo"][:,1] / np.sum(sf15["P_legs_wo"],axis=1)
P_30 = sf15["P_legs_wo"][:,2] / np.sum(sf15["P_legs_wo"],axis=1)
fig, ax = plt.subplots(1)
ax.plot(x,P_2/P_3,marker="x", linestyle="--", label="With CM")
ax.plot(x,P_20/P_30,marker="x", linestyle="--", label="Without CM")
ax.grid()
ax.legend()
ax.set_xlabel("$p_{null}$ [Pa]")
ax.set_ylabel("$P_2/P_3$")
ax.set_yscale("log")

In [ ]:
# Plot P_out vs. p_null
x = sf15["p_null"]
fig,ax = plt.subplots(1)
ax.plot(x,100*((sf15["P_out_w"]-sf15["P_out_wo"])/sf15["P_out_wo"]), marker="x", linestyle="--")
ax.set_ylabel(r"$\delta P_{out}$ [%]")
ax.set_xlabel("$p_{null}$ [Pa]")
ax.grid()

In [ ]:
runs = ["/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_1e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_2e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_3e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_4e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_5e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_6e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_7e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_8e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_9e19",
        "/Users/power8/Documents/07_churning_mode/n_scan_SF45/with_churn/n_10e19"]
sf45 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
# Plot P_out vs. p_null
x1 = sf15["p_null"]
x2 = sf45["p_null"]
fig,ax = plt.subplots(1)
ax.plot(x,((sf15["P_out_w"]-sf15["P_out_wo"])/sf15["P_out_wo"]), label="SF15")
ax.plot(x,((sf45["P_out_w"]-sf45["P_out_wo"])/sf45["P_out_wo"]), label="SF45")
ax.set_ylabel(r"$\delta P_{out}$ [%]")
ax.set_xlabel("$p_{null}$ [Pa]")
ax.grid()
ax.legend()

# 26/02/2025 analysis

In [ ]:
# beta_p heatmap
import matplotlib.colors as colors
# ds = read_boutdata(r"../data", remove_xgc = True, units="m")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_10/TCV_dxx_scan/with_churn/sigma_0p3", remove_xgc = True, units="m")
_,_,_,_,psi1,psi2 = find_null_coords(ds, timestep=0)

alpha = 0.4
x_r = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_r = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)
ds_r = ds.interp(x=x_r,y=y_r)

t = -1
fig,ax = plt.subplots(1, figsize=(6.5,3))
psi = ds_r["psi"].isel(t=t).transpose()
# beta_p = np.where(psi > 0, ds["beta_p"].isel(t=0).transpose(), 0.0)
beta_p = ds_r["beta_p"].isel(t=t).transpose()

ax.contour(ds_r.x,ds_r.y,psi, colors="black", linestyles="-", levels=[psi2])
ax.contour(ds_r.x,ds_r.y,psi, colors="black", linestyles="-", levels=[psi1])
c = ax.contourf(ds_r.x,ds_r.y,beta_p, extend="max", cmap="RdYlBu_r", levels=np.linspace(0.01,10,1000), norm=colors.LogNorm(vmin=0.01, vmax=10))
ax.contour(ds_r.x,ds_r.y,beta_p, colors="white", levels=[1,3,10])

ax.set_xlim([-0.14,0.21])
ax.set_ylim([-0.05,0.125])
ax.set_xticks([-0.15,-0.10,-0.05,0,0.05,0.10,0.15,0.20])
ax.set_yticks([-0.05,0,0.05,0.10])
fig.colorbar(c, ticks=[0.01, 0.1, 1, 10], label=r"$\beta_p$")
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
fig.tight_layout()

In [ ]:
# B_p heatmap
ds = read_boutdata(r"../data", remove_xgc = True, units="m")
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_7/TCV_dxx_scan/with_churn/sigma_0p3", remove_xgc = True, units="m")
ds["B_p"] = np.sqrt(ds["B_x"]**2 + ds["B_y"]**2) * ds.metadata["B_pmid"]
_,_,_,_,psi1,psi2 = find_null_coords(ds, timestep=0)
alpha = 0.45
x_r = ds.x * np.cos(alpha) - ds.y*np.sin(alpha)
y_r = ds.x * np.sin(alpha) + ds.y*np.cos(alpha)
ds_r = ds.interp(x=x_r,y=y_r)

fig,ax = plt.subplots(1)
c = ax.contourf(ds_r.x,ds_r.y,ds_r["B_p"].isel(t=-1).transpose()/0.2, cmap="RdYlBu_r", extend="both", levels=np.linspace(0,0.3,100))
ax.contour(ds_r.x,ds_r.y,ds_r["B_p"].isel(t=-1).transpose()/0.2, colors="red", linestyles="--", levels=[0.1])
ax.contour(ds_r.x,ds_r.y,ds_r["psi"].isel(t=-1).transpose(), colors="black", linestyles="-", levels=[psi1, psi2])
fig.colorbar(c)

In [ ]:
runs = ["/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_2p5cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_5p0cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_7p5cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_10p0cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_12p5cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_15p0cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_17p5cm",
        "/Users/power8/Documents/07_churning_mode/d_xx_scan_SF75/with_churn/d_xx_20p0cm"]
sf75 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
a_mid = 0.65
x = sf75["d_xx"] / a_mid
P_4 = sf75["P_legs_w"][:,3] / np.sum(sf75["P_legs_w"],axis=1)
P_2 = sf75["P_legs_w"][:,1] / np.sum(sf75["P_legs_w"],axis=1)
fig, ax = plt.subplots(1)
# ax.plot(x,100*P_1,marker="x", linestyle="--", label="Leg 4", color="blue")
# ax.plot(x,100*P_3,marker="x", linestyle="--", label="Leg 2", color="red")
ax.plot(x,P_2 / P_4, marker="x", linestyle="--", label="Leg 2", color="red")
ax.grid()
# ax.legend()
ax.set_xlabel("$\sigma$")
ax.set_ylabel("$P_2 / P_4$")

In [ ]:
runs = [r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p1",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p2",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p3",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p4",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p5",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_7p5/with_churn/sigma_0p6"]
chi7p5 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
runs = [r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p1",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p2",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p3",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p4",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p5",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_7p5/with_churn/sigma_0p6"]
chi7p5_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
# a_mid = 0.25
a_mid = 0.22
x = chi7p5["d_xx"] / a_mid
P_4 = chi7p5["P_legs_w"][:,3] / np.sum(chi7p5["P_legs_w"],axis=1)
P_2 = chi7p5["P_legs_w"][:,1] / np.sum(chi7p5["P_legs_w"],axis=1)
P_4ne = chi7p5_ne["P_legs_w"][:,3] / np.sum(chi7p5_ne["P_legs_w"],axis=1)
P_2ne = chi7p5_ne["P_legs_w"][:,1] / np.sum(chi7p5_ne["P_legs_w"],axis=1)
P_40 = chi7p5_ne["P_legs_wo"][:,3] / np.sum(chi7p5_ne["P_legs_wo"],axis=1)
P_20 = chi7p5_ne["P_legs_wo"][:,1] / np.sum(chi7p5_ne["P_legs_wo"],axis=1)
fig, ax = plt.subplots(1)
ax.plot(x,P_2 / P_4, marker="x", linestyle="--", label="ELM peak conditions", color="red")
ax.plot(x,P_2ne / P_4ne, marker="x", linestyle="--", label="L-mode like conditions", color="blue")
ax.plot(x,P_20 / P_40, marker="x", linestyle="--", label="No CM", color="gray")
ax.grid()
ax.legend()
ax.set_xlim([0,2.5])
ax.set_ylim([0,0.5])
ax.set_xlabel(r"$\sigma$")
ax.set_ylabel(r"$P_2 / P_4$")

In [ ]:
runs = [r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p1",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p2",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p3",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p4",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p5",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p6",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p7",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan/chi_perp_15p0/with_churn/sigma_0p8"]
chi15p0 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
runs = [r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p1",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p2",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p3",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p4",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p5",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p6",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p7",
        r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_dxx_scan_noELM/chi_perp_15p0/with_churn/sigma_0p8"]
chi15p0_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
# Plot P2 / P4 ratio vs sigma
# a_mid = 0.25
a_mid = 0.22
x = chi15p0["d_xx"] / a_mid
P_4 = chi15p0["P_legs_w"][:,3] / np.sum(chi15p0["P_legs_w"],axis=1)
P_2 = chi15p0["P_legs_w"][:,1] / np.sum(chi15p0["P_legs_w"],axis=1)
P_4ne = chi15p0_ne["P_legs_w"][:,3] / np.sum(chi15p0_ne["P_legs_w"],axis=1)
P_2ne = chi15p0_ne["P_legs_w"][:,1] / np.sum(chi15p0_ne["P_legs_w"],axis=1)
P_40 = chi15p0_ne["P_legs_wo"][:,3] / np.sum(chi15p0_ne["P_legs_wo"],axis=1)
P_20 = chi15p0_ne["P_legs_wo"][:,1] / np.sum(chi15p0_ne["P_legs_wo"],axis=1)
fig, ax = plt.subplots(1)
ax.plot(x,P_2 / P_4, marker="x", linestyle="--", label="ELM peak conditions", color="red")
ax.plot(x,P_2ne / P_4ne, marker="x", linestyle="--", label="L-mode like conditions", color="blue")
ax.plot(x,P_20 / P_40, marker="x", linestyle="--", label="No CM", color="gray")
ax.grid()
ax.legend()
ax.set_xlim([0,2.5])
ax.set_ylim([0,0.5])
ax.set_xlabel(r"$\sigma$")
ax.set_ylabel(r"$P_2 / P_4$")

In [ ]:
sigmas = ["sigma_0p1", "sigma_0p2", "sigma_0p3", "sigma_0p4", "sigma_0p5", "sigma_0p6", "sigma_0p7", "sigma_0p8"]

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_2/TCV_dxx_scan/chi_perp_10p0/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv2_10 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_2/TCV_dxx_scan/chi_perp_15p0/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv2_15 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_3/TCV_dxx_scan_noELM/chi_perp_1p0/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv3_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_4/TCV_dxx_scan/chi_perp_10p0/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv4_10 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_4/TCV_dxx_scan/chi_perp_15p0/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv4_15 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_5/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv5 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_6/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv6 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_7/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv7 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_7/TCV_dxx_scan_noELM/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv7_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_8/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv8 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_9/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv9 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/TCV_dxx_scans/TCV_scans_9/TCV_dxx_scan_noELM/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv9_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
sigmas10 = ["sigma_0p1", "sigma_0p2", "sigma_0p3", "sigma_0p4", "sigma_0p5", "sigma_0p6", "sigma_0p7", "sigma_0p8", "sigma_0p9", "sigma_1p0"]
rundir = "/Users/power8/Documents/07_churning_mode/TCV_dxx_scans/TCV_scans_10/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas10]
tcv10 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/TCV_dxx_scans/TCV_scans_10/TCV_dxx_scan_noELM/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv10_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_10_convedit/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv10_convedit = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_11/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv11 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_11/TCV_dxx_scan_noELM/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tcv11_ne = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
# Plot P2 / P4 ratio vs sigma
plot_P2_P4_ratio(tcv10, tcv10_ne, show_no_cm=False, a_mid=0.25, trim_first=0)

In [ ]:
plot_P2_P4(tcv9, a_mid=0.25)

In [ ]:
plot_P1_plus_P3(tcv10, a_mid=0.25)

In [ ]:
# Compare P2/P4 at different B_pm
fig,ax = plt.subplots(1)
a_mid = 0.25
x1 = tcv7["d_xx"] / a_mid
x2 = tcv10["d_xx"] / a_mid
x3 = tcv9["d_xx"] / a_mid
y1 = tcv7["P_legs_w"][:,1] / tcv7["P_legs_w"][:,3]
y2 = tcv10["P_legs_w"][:,1] / tcv10["P_legs_w"][:,3]
y3 = tcv9["P_legs_w"][:,1] / tcv9["P_legs_w"][:,3]
ax.plot(x1,y1, linestyle="-", color="gray")
ax.plot(x2,y2, linestyle="--", marker="x", color="red", label=r"$B_{pm}$")
ax.plot(x3,y3, linestyle="-", color="gray", label=r"$B_{pm}\pm 0.01T$")
ax.fill_between(x1,y1,y3, alpha=0.5)
ax.legend()
ax.grid()
ax.set_ylabel(r"$P_2 / P_4$")
ax.set_xlabel(r"$\sigma$")
fig.tight_layout()

# 05/03/2025 analysis

In [ ]:
# ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_10/TCV_dxx_scan/with_churn/sigma_0p1", remove_xgc = True, units="cm")
explore_nulls(ds, t=10, vmin=0, vmax=3, levels=200)

In [ ]:
psi_0 = 0.0
psi_a = 1.0
t = np.array([0,10,20,30,40,50,60,70,80,90,100])
psi_0_sigma_0p1 = np.array([0,-0.003,0.00014,-0.00452,0.00503,0.01118,-0.00307,-0.00084,0.00084,-0.005,-0.00619])
rho_0_sigma_0p1 = np.sqrt((psi_0_sigma_0p1 - psi_a) / (psi_0 - psi_a))
diff_min = -2.957
diff_max = 4.58
fig,ax = plt.subplots(1)
x = 1000*(ds.t.isel(t=t) - ds.t.values[0]) * ds.metadata["t_0"]
ax.plot(x,rho_0_sigma_0p1)
ax.grid()
ax.set_xlabel("t [ms]")
ax.set_ylabel(r"$\rho_{sepx} = \sqrt{\phi_{N,sepx}}$")

In [ ]:
ds["rho"] = np.sqrt((ds["psi"] - psi_a) / (psi_0 - psi_a))
fig,ax = plt.subplots(1)
ax.plot(ds.x,ds["rho"].isel(t=0,y=-1))
for rho_0 in rho_0_sigma_0p1:
    ax.axvline(rho_0,linestyle="--")

# 19/03/2025 analysis

In [ ]:
sigmas = ["sigma_0p1", "sigma_0p3", "sigma_0p5", "sigma_0p8"]
sigmas_full =  ["sigma_0p1", "sigma_0p3", "sigma_0p4", "sigma_0p5", "sigma_0p6", "sigma_0p7", "sigma_0p8", "sigma_1p0"]

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/TCV_dxx_scans/TCV_scans_10/TCV_dxx_scan/with_churn"
runs = [rundir + "/" + s for s in sigmas]
orig = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/both_fwd/with_churn"
runs = [rundir + "/" + s for s in sigmas]
fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/both_rev/with_churn"
runs = [rundir + "/" + s for s in sigmas]
rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_fwd/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tfonly_fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_rev/with_churn"
runs = [rundir + "/" + s for s in sigmas]
tfonly_rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs_highb/both_fwd/with_churn"
runs = [rundir + "/" + s for s in sigmas_full]
fwd2 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs_highb/both_rev/with_churn"
runs = [rundir + "/" + s for s in sigmas_full]
rev2 = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=10)

In [ ]:
trim_first = 1
a_mid = 0.25
fig, ax = plt.subplots(1)
label = ["CM only", r"$\nabla_\parallel p$ effects only, FWD $B_{t0}$", r"$\nabla_\parallel p$ effects only, REV $B_{t0}$"]
for i,data in enumerate([orig, tfonly_fwd, tfonly_rev]):
    x = data["d_xx"][trim_first:] / a_mid
    P_4 = data["P_legs_w"][trim_first:, 3] / np.sum(
        data["P_legs_w"][trim_first:], axis=1
    )
    P_2 = data["P_legs_w"][trim_first:, 1] / np.sum(
        data["P_legs_w"][trim_first:], axis=1
    )
    P_40 = data["P_legs_wo"][trim_first:, 3] / np.sum(
        data["P_legs_wo"][trim_first:], axis=1
    )
    P_20 = data["P_legs_wo"][trim_first:, 1] / np.sum(
        data["P_legs_wo"][trim_first:], axis=1
    )
    ax.plot(
        x,
        P_2 / P_4,
        marker="x",
        linestyle="--",
        label=label[i],
        # color="red",
        zorder=999,
    )
    ax.grid()
    ax.legend()
    ax.set_xlim([0,2.5])
    ax.set_ylim([0,0.5])
    ax.set_xlabel(r"$\sigma$")
    ax.set_ylabel(r"$P_2 / P_4$")

In [ ]:
plot_P2_P4_ratio(orig, show_no_cm=True, a_mid=0.25, trim_first=0, ylim=[0,0.5])
plot_P2_P4_ratio(tfonly_fwd, show_no_cm=True, a_mid=0.25, trim_first=0, ylim=[0,0.5])
plot_P2_P4_ratio(tfonly_rev, show_no_cm=True, a_mid=0.25, trim_first=0, ylim=[0,0.5])

In [ ]:
plot_P2_P4(orig, a_mid=0.25)
# plot_P2_P4(fwd2, a_mid=0.25)
plot_P2_P4(rev2, a_mid=0.25)

In [ ]:
ax = plot_P1_plus_P3(orig, a_mid=0.25, label="CM only")
plot_P1_plus_P3(tfonly_fwd, a_mid=0.25, ax=ax, label="FWD")
plot_P1_plus_P3(tfonly_rev, a_mid=0.25, ax=ax, label="REV")

In [ ]:
ax = plot_f_inner_outer(orig, a_mid=0.25, label="CM only")
plot_f_inner_outer(tfonly_fwd, a_mid=0.25, ax=ax, label="FWD")
plot_f_inner_outer(tfonly_rev, a_mid=0.25, ax=ax, label='REV')

# 27/03/2025 analysis

In [ ]:
# sigmas = ["sigma_0p1", "sigma_0p3", "sigma_0p5", "sigma_0p8"]

# rundir = "/Users/power8/Documents/07_churning_mode/TCV_dxx_scans/TCV_scans_10/TCV_dxx_scan/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# cmonly = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_fwd/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_rev/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_fwd_new/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_fwd_new = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/thermal_force_runs/tfonly_rev_new/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_rev_new = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=25)


In [ ]:
sigmas = ["sigma_0p1","sigma_0p3", "sigma_0p2", "sigma_0p4", "sigma_0p5", "sigma_0p6", "sigma_0p7", "sigma_0p8", "sigma_1p0", "sigma_1p2"]

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_12/H/with_churn"
runs = [rundir + "/" + s for s in sigmas]
cmonly = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=20)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_12/H/with_tfonly_fwd"
runs = [rundir + "/" + s for s in sigmas]
tfonly_fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=20)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_12/H/with_tfonly_rev"
runs = [rundir + "/" + s for s in sigmas]
tfonly_rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=20)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_12/H/with_both_fwd"
runs = [rundir + "/" + s for s in sigmas]
both_fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=20)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_12/H/with_both_rev"
runs = [rundir + "/" + s for s in sigmas]
both_rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=20)

In [ ]:
sigmas = ["sigma_0p4", "sigma_0p5", "sigma_0p6", "sigma_0p7"]

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_13/with_churn"
# runs = [rundir + "/" + s for s in sigmas]
# cmonly = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=5)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_13/with_tfonly_fwd"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_fwd = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=5)

# rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_13/with_tfonly_rev"
# runs = [rundir + "/" + s for s in sigmas]
# tfonly_rev = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=5)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_13/with_both_fwd"
runs = [rundir + "/" + s for s in sigmas]
both_fwd_sh = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/TCV_scans_13/with_both_rev"
runs = [rundir + "/" + s for s in sigmas]
both_rev_sh = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=50)

In [ ]:
trim_first = 0
a_mid = 0.20
fig, ax = plt.subplots(1)
# label = ["CM only", r"$\nabla_\parallel p$ effects only, FWD $B_{t0}$", r"$\nabla_\parallel p$ effects only, REV $B_{t0}$"]
label = ["Both FWD", "Both REV"]
# for i,data in enumerate([cmonly, tfonly_fwd, tfonly_rev]):
# for i,data in enumerate([both_fwd, both_rev]):
for i,data in enumerate([both_fwd_sh, both_rev_sh]):
    x = data["d_xx"][trim_first:] / a_mid
    P_4 = data["P_legs_w"][trim_first:, 3] / np.sum(
        data["P_legs_w"][trim_first:], axis=1
    )
    P_2 = data["P_legs_w"][trim_first:, 1] / np.sum(
        data["P_legs_w"][trim_first:], axis=1
    )
    P_40 = data["P_legs_wo"][trim_first:, 3] / np.sum(
        data["P_legs_wo"][trim_first:], axis=1
    )
    P_20 = data["P_legs_wo"][trim_first:, 1] / np.sum(
        data["P_legs_wo"][trim_first:], axis=1
    )
    ax.plot(
        x,
        P_2 / P_4,
        marker="x",
        linestyle="--",
        label=label[i],
        # color="red",
        zorder=999,
    )
    # ax.plot(
    #     x,
    #     P_20 / P_40,
    #     marker="x",
    #     linestyle="--",
    #     # label=label[i],
    #     color="gray",
    #     zorder=999,
    # )
ax.legend()
ax.set_xlim([-0.02,2.5])
ax.set_ylim([-0.01,0.5])
ax.set_xlabel(r"$\sigma$")
ax.set_ylabel(r"$P_2 / P_4$")
ax.grid()

In [ ]:
plot_P2_P4(both_fwd_sh, a_mid=0.20)
plot_P2_P4(both_rev_sh, a_mid=0.20)

In [ ]:
# ax = plot_P1_plus_P3(cmonly, a_mid=0.20, label="CM only")
ax = plot_P1_plus_P3(both_fwd_sh, a_mid=0.20, label="FWD")
plot_P1_plus_P3(both_rev_sh, a_mid=0.20, ax=ax, label="REV")

# 09/04/2024 analysis

In [ ]:
densities = ["n1","n2","n3","n4","n5","n6","n7","n8"]
rundir = "/Users/power8/Documents/07_churning_mode/SCRATCH/fit_runs/TCV/n_scan/with_churn"
runs = [rundir + "/" + d for d in densities]
nscan = extract_rundeck_data(runs=runs, d_sol=0.1, avg_window=15)

In [ ]:
nscan["r_ch"] * 100

In [ ]:
# P_out vs. p_null

fig,ax = plt.subplots(1, figsize=(4.0,2.5))

ax.plot(100*nscan["r_ch"], 1e-6 * nscan["P_out_w"], color="red", marker="x", linestyle="--", label="With CM")
ax.plot(100*nscan["r_ch"], 1e-6 * nscan["P_out_wo"], color="black", marker="x", linestyle="--", label="Without CM")
ax.set_xlabel("$r_{cz}$ [cm]")
ax.set_ylabel("$P_{out}$ [MWm$^{-1}$]")
ax.grid()
fig.tight_layout()
ax.legend()

# 06/04/2025 analysis

In [ ]:
ds = read_boutdata(r"/Users/power8/Documents/07_churning_mode/SCRATCH/MASTU_runs/n_scan/with_churn/n8", remove_xgc = True, units="a_mid")
